In [18]:
# import statements
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import glob
import os

In [64]:


# importing a sample
arr = pd.read_csv('testWave005_mijn35_R001.csv', header=None, delimiter=";")
# check the shape
input_shape = arr.shape
print(arr.iloc[0])
print(arr.iloc[1])
print(f"The input shape of your array is: {input_shape}")

0                                     Frame
1              Elapsed_Time_In_Milliseconds
2                                      Time
3                                    Hand_X
4                                    Hand_Y
                       ...                 
362       Ring_DIP_Flex_AngularAcceleration
363    Pinky_MCP_Spread_AngularAcceleration
364      Pinky_MCP_Flex_AngularAcceleration
365      Pinky_PIP_Flex_AngularAcceleration
366      Pinky_DIP_Flex_AngularAcceleration
Name: 0, Length: 367, dtype: object
0                 0
1                 0
2      00:00:00:000
3                 0
4                 0
           ...     
362        37274,14
363       -46771,95
364        27792,87
365        87709,06
366        83074,34
Name: 1, Length: 367, dtype: object
The input shape of your array is: (283, 367)


In [65]:
folder_path = 'csv_letters/*.csv'
x = []
y = []

# Example class names
class_names = ["letterA", "letterB", "letterC","letterD","letterE","letterF","letterG","letterH","letterI","letterK","letterL","letterM","letterN","letterO","letterP","letterQ","letterR","letterS","letterT","letterU","letterV","letterW","letterX","letterY"]

# Create a dictionary mapping class names to numerical labels
class_to_label = {class_name: label for label, class_name in enumerate(class_names)}
# Use glob to get all the csv files in the folder
csv_files = glob.glob(folder_path)

# Initialize an empty list to store the combined DataFrames
combined_dataframes = []
max_len = 0
for i in range(1, len(csv_files), 2):
    # Read the first file into a DataFrame
    df1 = pd.read_csv(csv_files[i], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal=',')
    if (len(df1)>max_len):
        max_len = len(df1)
print(f"max length is {max_len}")

max length is 315


In [66]:

# Iterate over the files two by two
for i in range(1, len(csv_files), 2):
    # Read the first file into a DataFrame
    df1 = pd.read_csv(csv_files[i], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal=',')
    # Get the shape of df1
    # print(f"Shape of df1: {df1.shape}")

    #get the class and add it to y
    name = os.path.splitext(csv_files[i])
    classtype = name[0].split('\\')[1][:7] #this takes letterX from the title
    numerical_label = class_to_label.get(classtype, -1)  # -1 if not found
    if numerical_label != -1:
        y.append(numerical_label)
    else:
        print("failed to add to y: " + classtype)

    # If there is a next file, read it and combine with the first
    if i+1 < len(csv_files):
        df2 = pd.read_csv(csv_files[i+1], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal=',')
        df2 = df2.add_prefix('right_')
    else:
        # If there is no next file, create an empty DataFrame with the same columns as df1
        df2 = pd.DataFrame(columns=df1.columns)

    # Ensure that both DataFrames have the same number of rows by filling NaN values in df2
    if len(df1) > len(df2):
        df2 = df2.reindex(df1.index, fill_value=np.nan)
    elif len(df2) > len(df1):
        df1 = df1.reindex(df2.index, fill_value=np.nan)

    # Concatenate the DataFrames
    combined_df = pd.concat([df1, df2], axis=1)
    combined_df = combined_df.reindex(range(max_len)).fillna(0)
    # Print the shape of combined_df
    print(f"Shape of combined_df: {combined_df.shape}")
    combined_dataframes.append(combined_df)

# Convert the list of DataFrames to a NumPy array
x = np.array(combined_dataframes)
x = np.asarray(x).astype('float32') # meant to fix an error when training the model
y = np.array(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# print(x)
# print(y)



Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of c

In [67]:
model = models.Sequential()
model.add(layers.Conv1D(64, 3, activation='relu', input_shape=(315,732)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(24, activation='softmax'))

In [68]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_18 (Conv1D)          (None, 313, 64)           140608    
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 156, 64)          0         
 g1D)                                                            
                                                                 
 conv1d_19 (Conv1D)          (None, 154, 128)          24704     
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 77, 128)          0         
 g1D)                                                            
                                                                 
 conv1d_20 (Conv1D)          (None, 75, 128)           49280     
                                                                 
 flatten_6 (Flatten)         (None, 9600)             

In [69]:
model.compile(optimizer='adam',  # we used adam during machine learning course
              loss='sparse_categorical_crossentropy',  # Multiclass classification loss
              metrics=['accuracy'])  # Track accuracy during training
print(x.size)
print(y.size)
training_data = model.fit(x_train,y_train,epochs=50)

58797900
255
Epoch 1/50
7/7 [==============================] - 1s 33ms/step - loss: 729.7145 - accuracy: 0.0539
Epoch 2/50
7/7 [==============================] - 0s 29ms/step - loss: 114.8136 - accuracy: 0.1373
Epoch 3/50
7/7 [==============================] - 0s 30ms/step - loss: 29.7235 - accuracy: 0.2941
Epoch 4/50
7/7 [==============================] - 0s 29ms/step - loss: 11.9651 - accuracy: 0.4412
Epoch 5/50
7/7 [==============================] - 0s 26ms/step - loss: 5.0993 - accuracy: 0.5686
Epoch 6/50
7/7 [==============================] - 0s 29ms/step - loss: 2.2543 - accuracy: 0.7451
Epoch 7/50
7/7 [==============================] - 0s 27ms/step - loss: 0.7723 - accuracy: 0.8235
Epoch 8/50
7/7 [==============================] - 0s 29ms/step - loss: 0.4837 - accuracy: 0.8824
Epoch 9/50
7/7 [==============================] - 0s 27ms/step - loss: 0.3388 - accuracy: 0.9314
Epoch 10/50
7/7 [==============================] - 0s 27ms/step - loss: 0.2218 - accuracy: 0.9412
Epoch 11/5

In [70]:
result = model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 17ms/step - loss: 26.4745 - accuracy: 0.1569


In [24]:
import os
import pandas as pd

# Specify the folder path where your CSV files are located
folder_path = 'csv_letters_filtered'

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Iterate through each CSV file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)

        # Read the first 50 rows from the CSV file
        cols_to_use = list(range(3, 367))
        df = pd.read_csv(file_path,header=0, nrows=50,delimiter=';',usecols=cols_to_use,decimal=',', dtype=float)

        # Add a new column with the filename
        df['Original_Filename'] = filename
        # Append the data to the combined DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# Save the combined data to a new CSV file
combined_df.to_csv('combined_data.csv', index=False)

print("Current working directory:", os.getcwd())


ValueError: Usecols do not match columns, columns expected but not found: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208,